In [ ]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Upload the file
uploaded = files.upload()

# Get the filename from the uploaded files
filename = next(iter(uploaded))

# Read the contents of the uploaded file
file_content = uploaded[filename]

# Load the JSON data
intents = json.loads(file_content)

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')

Saving intents.json to intents (17).json
Epoch 1/200
6/6 [==============================] - 1s 5ms/step - loss: 1.4173 - accuracy: 0.1923
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2588 - accuracy: 0.5000
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.2830 - accuracy: 0.6154
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3024 - accuracy: 0.3846
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2417 - accuracy: 0.4615
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1525 - accuracy: 0.5385
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.1706 - accuracy: 0.5385
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1139 - accuracy: 0.5000
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.0124 - accuracy: 0.6154
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 0.9046 - accur

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [94]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model
lemmatizer = WordNetLemmatizer()

# Upload the file
uploaded = files.upload()

# Get the filename from the uploaded files
filename = next(iter(uploaded))

# Read the contents of the uploaded file
file_content = uploaded[filename]

# Load the JSON data
intents = json.loads(file_content)

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('/content/chatbot_model.h5')
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1],reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intent = intents_json['intents']
  for i in list_of_intent:
    if i['tag']==tag:
      result = random.choice(i['responses'])
      break
  return result

print("Go")
while True:
  message = input("hi")
  ints = predict_class(message)
  res = get_response(ints,intents)
  print(res)



Saving intents.json to intents (21).json
Go
hihi
1/1 [==============================] - 0s 71ms/step
Hello!
hiWhat is progamming
1/1 [==============================] - 0s 23ms/step
Programming, coding or software development, means writing computer code to automate tasks.
hiBest way to learn to code
1/1 [==============================] - 0s 21ms/step
Check out the NeuralNine YouTube channel and The Python Bible series (7 in 1).


KeyboardInterrupt: Interrupted by user